In [1]:
import time
import random
import math

In [2]:
people = [('Seymour', 'BOS'),
          ('Franny', 'DAL'),
          ('Zooey', 'CAK'),
          ('Walt', 'MIA'),
          ('Buddy', 'ORD'),
          ('Les', 'OMA')]

destination = 'LGA'

In [12]:
flights = {}

for line in open('schedule.txt'):
    origin, dest, depart, arrive, price = line.strip().split(',')
    flights.setdefault((origin, dest), [])
    # 将航班详情添加到航班列表中
    flights[(origin, dest)].append((depart, arrive, int(price)))

# 获取分钟数
def getminutes(t):
    x = time.strptime(t, '%H:%M')
    return x[3]*60 +x[4]

# 打印时间表
def printschedule(r):
    for d in range(int(len(r)/2)):
        name = people[d][0]
        origin = people[d][1]
        out = flights[(origin, destination)][r[2*d]]
        ret = flights[(destination, origin)][r[2*d+1]]
        print('%10s%10s %5s-%5s $%3s %5s-%5s $%3s' % (name, origin, out[0], out[1], out[2], ret[0], ret[1], ret[2]))

In [4]:
# 成本函数
def schedulecost(sol):
    totalprice = 0
    latestarrival = 0
    earliestdep = 24*60
    
    for d in range(int(len(sol)/2)):
        # 得到往返航班
        origin = people[d][1]
        outbound = flights[(origin, destination)][int(sol[2*d])]
        returnf = flights[(destination, origin)][int(sol[2*d+1])]
        
        # 总价格等于所有往返航班价格之和
        totalprice += outbound[2]
        totalprice += returnf[2]
        
        # 记录最晚到达时间和最早离开时间
        if latestarrival < getminutes(outbound[1]):
            latestarrival = getminutes(outbound[1])
        if earliestdep > getminutes(returnf[0]):
            earliestdep = getminutes(returnf[0])
    
    # 每个人必须在机场等待，直到最后一个人到达
    # 每个人必须在相同时间到达机场，等候往返航班
    # 在机场的等待时间价值1美元
    totalwait = 0
    for d in range(int(len(sol)/2)):
        origin = people[d][1]
        outbound = flights[(origin, destination)][int(sol[2*d])]
        returnf = flights[(destination, origin)][int(sol[2*d+1])]
        totalwait += latestarrival - getminutes(outbound[1])
        totalwait += getminutes(returnf[0]) - earliestdep
        
    if latestarrival < earliestdep:
        totalprice += 50
    
    return totalprice+totalwait

In [5]:
s = [1,4,3,2,7,3,6,3,2,4,5,3]
print(schedulecost(s))
printschedule(s)

4585
   Seymour       BOS  8:04-10:11 $ 95 12:08-14:05 $142
    Franny       DAL 10:30-14:57 $290  9:49-13:51 $229
     Zooey       CAK 17:08-19:08 $262 10:32-13:16 $139
      Walt       MIA 15:34-18:11 $326 11:08-14:38 $262
     Buddy       ORD  9:42-11:32 $169 12:08-14:47 $231
       Les       OMA 13:37-15:08 $250 11:07-13:24 $171


In [6]:
# 随机搜索
def randomoptimize(domain, costf):
    best = 99999999
    bestr = None
    for i in range(1000):
        # 创建一个随机解
        r = [random.randint(domain[i][0], domain[i][1]) for i in range(len(domain))]
        # 得到成本
        cost = costf(r)
        
        if cost < best:
            best = cost
            bestr = r
    return r

In [7]:
domain = [(0,9)] * (len(people) * 2)
ss = randomoptimize(domain, schedulecost)
print(schedulecost(ss))
printschedule(ss)

7598
   Seymour       BOS 13:40-15:37 $138 17:03-18:03 $103
    Franny       DAL  9:08-12:12 $364 15:49-20:10 $497
     Zooey       CAK 20:30-23:11 $114  8:19-11:16 $122
      Walt       MIA 14:01-17:24 $338 15:23-18:49 $150
     Buddy       ORD 11:01-12:39 $260 14:19-17:09 $190
       Les       OMA  7:39-10:24 $219 12:31-14:02 $234


In [8]:
# 爬山法
# 基于时间成本，寻找更合适的相邻安排
def hillclimb(domain, costf):
    # 创建一个随机解
    sol = [random.randint(domain[i][0], domain[i][1]) for i in range(len(domain))]
    
    while True:
        # 创建相邻解的列表
        neighbors = []
        for j in range(len(domain)):
            # 在每个方向上偏离原值一点
            if sol[j] > domain[j][0]:
                neighbors.append(sol[0:j] + [sol[j]-1] + sol[j+1:])
            
            if sol[j] < domain[j][1]:
                neighbors.append(sol[0:j] + [sol[j]+1] + sol[j+1:])
                
        current = costf(sol)
        best = current
        for j in range(len(neighbors)):
            cost = costf(neighbors[j])
            if cost < best:
                best = cost
                sol = neighbors[j]
        
        if best == current:
            break
            
    return sol

In [9]:
ss = hillclimb(domain, schedulecost)
print(schedulecost(ss))
printschedule(ss)

3464
   Seymour       BOS 17:11-18:30 $108 13:39-15:30 $ 74
    Franny       DAL 13:54-18:02 $294 17:14-20:59 $277
     Zooey       CAK 15:23-17:25 $232 13:37-15:33 $142
      Walt       MIA 15:34-18:11 $326 15:23-18:49 $150
     Buddy       ORD  9:42-11:32 $169 14:19-17:09 $190
       Les       OMA 12:18-14:56 $172 15:07-17:21 $129


In [14]:
def annealingoptimize(domain, costf, T=10000, cool=0.95, step=1):
    # 随机初始化值
    vec = [float(random.randint(domain[i][0], domain[i][1])) for i in range(len(domain))]
    
    while T > 0.1:
        # 随机选择一个索引值
        i = random.randint(0, len(domain)-1)
        # 随机选择一个改变索引值的方向
        dir = random.randint(-step, step)
        # 创建一个代表题解的新列表，改变其中一个值
        vecb = vec[:]
        vecb[i] += dir
        if vecb[i] < domain[i][0]:
            vecb[i] = domain[i][0]
        elif vecb[i] > domain[i][1]:
            vecb[i] = domain[i][1]
        # 计算当前成本和新的成本
        ea = costf(vec)
        eb = costf(vecb)
        # 模拟退火
        if (eb < ea or random.random() < pow(math.e, -(eb-ea)/T)):
            vec = vecb
        # 降低温度
        T = T*cool
    
    vec = [int(i) for i in vec]
    return vec

In [15]:
ss = annealingoptimize(domain, schedulecost)
print(schedulecost(ss))
printschedule(ss)

3273
   Seymour       BOS 13:40-15:37 $138 17:03-18:03 $103
    Franny       DAL 10:30-14:57 $290 14:20-17:32 $332
     Zooey       CAK 13:40-15:38 $137 13:37-15:33 $142
      Walt       MIA 11:28-14:40 $248 12:37-15:05 $170
     Buddy       ORD 12:44-14:17 $134 12:08-14:47 $231
       Les       OMA  9:15-12:03 $ 99 15:07-17:21 $129


In [49]:
def geneticoptimize(domain,costf,popsize=50,step=1,
                    mutprob=0.2,elite=0.2,maxiter=100):
    # Mutation Operation
    def mutate(vec):
        i=random.randint(0,len(domain)-1)
        if random.random()<0.5 and vec[i]>domain[i][0]:
            return vec[0:i]+[vec[i]-step]+vec[i+1:] 
        elif vec[i]<domain[i][1]:
            return vec[0:i]+[vec[i]+step]+vec[i+1:]
        else:
            return vec[0:i]+[vec[i]-step]+vec[i+1:]
  
    # Crossover Operation
    def crossover(r1,r2):
        i=random.randint(1,len(domain)-2)
        return r1[0:i]+r2[i:]

    # Build the initial population
    pop=[]
    for i in range(popsize):
        vec=[random.randint(domain[i][0],domain[i][1]) for i in range(len(domain))]
        pop.append(vec)
  
    # How many winners from each generation?
    topelite=int(elite*popsize)
  
    # Main loop 
    for i in range(maxiter):
        scores=[(costf(v),v) for v in pop]
        scores.sort()
        ranked=[v for (s,v) in scores]

        # Start with the pure winners
        pop=ranked[0:topelite]

        # Add mutated and bred forms of the winners
        while len(pop)<popsize:
            if random.random()<mutprob:
                # Mutation
                c = random.randint(0,topelite)
                pop.append(mutate(ranked[c]))
                if pop[-1] == None:
                    print('mutprob', 'i', '!!')
                    print('test', test)
                    printnt('test1', test1)
            else:
                # Crossover
                c1=random.randint(0,topelite)
                c2=random.randint(0,topelite)
                pop.append(crossover(ranked[c1],ranked[c2]))
                if pop[-1] == None:
                    print('crossover', 'i', '!!')

        # Print current best score
        print(scores[0][0])
    
    return scores[0][1]

In [50]:
ss = geneticoptimize(domain, schedulecost)
print(schedulecost(ss))
printschedule(ss)

3122
2928
2842
2706
2687
2625
2553
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
2472
   Seymour       BOS 12:34-15:02 $109  6:39- 8:09 $ 86
    Franny       DAL 10:30-14:57 $290  6:09- 9:49 $414
     Zooey       CAK 12:08-14:59 $149  8:19-11:16 $122
      Walt       MIA 11:28-14:40 $248  6:33- 9:14 $172
     Buddy       ORD  8:25-10:34 $157  6:03- 8:43 $219


In [26]:
# The dorms, each of which has two available spaces
dorms=['Zeus','Athena','Hercules','Bacchus','Pluto']

# People, along with their first and second choices
prefs=[('Toby', ('Bacchus', 'Hercules')),
       ('Steve', ('Zeus', 'Pluto')),
       ('Karen', ('Athena', 'Zeus')),
       ('Sarah', ('Zeus', 'Pluto')),
       ('Dave', ('Athena', 'Bacchus')), 
       ('Jeff', ('Hercules', 'Pluto')), 
       ('Fred', ('Pluto', 'Athena')), 
       ('Suzie', ('Bacchus', 'Hercules')), 
       ('Laura', ('Bacchus', 'Hercules')), 
       ('James', ('Hercules', 'Athena'))]

In [27]:
domain = [(0, len(dorms)*2-i-1) for i in range(0, len(dorms)*2)]

def printsolution(vec):
    slots = []
    # 为每个宿舍建两个槽
    for i in range(len(dorms)):
        slots += [i,i]
        
    # 遍历每一名学生的安置情况
    for i in range(len(vec)):
        x = int(vec[i])
        
        # 从剩余槽中选择
        dorm = dorms[slots[x]]
        # 输出学生及其被分配的宿舍
        print(prefs[i][0], dorm)
        # 删除该槽
        del slots[x]

In [28]:
printsolution([0,0,0,0,0,0,0,0,0,0])

Toby Zeus
Steve Zeus
Karen Athena
Sarah Athena
Dave Hercules
Jeff Hercules
Fred Bacchus
Suzie Bacchus
Laura Pluto
James Pluto


In [29]:
def dormcost(vec):
    cost = 0
    # 建立一个槽序列
    slots = [0,0,1,1,2,2,3,3,4,4]
    # 遍历每一名学生
    for i in range(len(vec)):
        x = int(vec[i])
        dorm = dorms[slots[x]]
        pref = prefs[i][1]
        # 首选成本值为0，次选成本值为1
        if pref[0] == dorm:
            cost += 0
        elif pref[1] == dorm:
            cost += 1
        # 不在选择之列则成本值为3
        else:
            cost += 3
        # 删除选中的槽
        del slots[x]
        
    return cost

In [51]:
ss = randomoptimize(domain, dormcost)
print(dormcost(ss))
geneticoptimize(domain, dormcost)
printsolution(ss)

19
7
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
Toby Zeus
Steve Athena
Karen Athena
Sarah Hercules
Dave Zeus
Jeff Bacchus
Fred Pluto
Suzie Bacchus
Laura Hercules
James Pluto


In [52]:
people=['Charlie','Augustus','Veruca','Violet','Mike','Joe','Willy','Miranda']

links=[('Augustus', 'Willy'), 
       ('Mike', 'Joe'), 
       ('Miranda', 'Mike'), 
       ('Violet', 'Augustus'), 
       ('Miranda', 'Willy'), 
       ('Charlie', 'Mike'), 
       ('Veruca', 'Joe'), 
       ('Miranda', 'Augustus'), 
       ('Willy', 'Augustus'), 
       ('Joe', 'Charlie'), 
       ('Veruca', 'Augustus'), 
       ('Miranda', 'Joe')]

In [53]:
def crosscount(v):
    # 将数字序列转换成一个person:(x,y)的字典
    loc = dict([(people[i], (v[i*2], v[i*2+1])) for i in range(0, len(people))])
    total = 0
    # 遍历每一对连线
    for i in range(len(links)):
        for j in range(i+1, len(links)):
            # 获取坐标位置
            (x1, y1), (x2, y2) = loc[links[i][0]], loc[links[i][1]]
            (x3, y3), (x4, y4) = loc[links[j][0]], loc[links[j][1]]
            # 计算分数值
            den = (y4-y3)*(x2-x1) - (x4-x3)*(x2-x1)
            # 如果两线平行，den == 0
            if den == 0:
                continue
            # 否则，ua 与 ub 就是两条交叉线的分数值
            ua=((x4-x3)*(y1-y3)-(y4-y3)*(x1-x3))/den
            ub=((x2-x1)*(y1-y3)-(y2-y1)*(x1-x3))/den
            
            if ua>0 and ua<1 and ub>0 and ub<1:
                total+=1
                
    return total

In [54]:
domain = [(10, 370)] * (len(people)*2)

In [57]:
sol = randomoptimize(domain, crosscount)
print(crosscount(sol))

sol = annealingoptimize(domain, crosscount, step=50, cool=0.99)
print(crosscount(sol))
print(sol)

10
2
[11, 10, 37, 134, 235, 137, 215, 359, 138, 357, 10, 22, 21, 184, 24, 297]
